In [1]:
from collections import namedtuple

import itertools as itt
from tqdm import tnrange,tqdm_notebook
import re, struct
from ipywidgets import interact

from funcs import gps_coord, gln_coord, get_epoch_array, file_byte_iterator,  filter_epoch, general_parser, map_rec, Uchet

%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Category10
from scipy import signal
from bokeh.transform import jitter
TOOLS = "pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,save"
output_notebook()

Loading BokehJS ...

In [3]:
#30082018
#dx,dy,dz = 1.39394, 0.79917, -1.14693
#dx,dy,dz = 1.219, -0.36045, -0.61941
#dx,dy,dz = 0.15215, 1.30038, -0.66836

#13032018
#dx,dy,dz = 0.969724, -0.255055, -0.377584
#dx,dy,dz = 0.645298, 0.656907, -0.608911
#dx,dy,dz = 1.62673, 0.378355, -1.04789

#04052018
#dx,dy,dz = 0.9514162, -0.345039, -0.3368897
#dx,dy,dz = 0.7180163, 0.5830234, -0.6095884
#dx,dy,dz = 1.675363, 0.2086571, -1.018543

In [4]:
filename = 'data/rks.dat'
data = bytes(file_byte_iterator(filename))

In [5]:
epoch_array = get_epoch_array(data)
req_ids = [b'~~', b'SI', b'AN', b'rc', b'pc', b'RC', b'PC', b'GE', b'NE', b'PV', b'GT', b'NN', b'NT']
epoch_min_arr = [filter_epoch(req_ids, epoch) for epoch in epoch_array]

In [6]:
pseudo_params = namedtuple('pseudo_params', ['channel', 'usi', 'scp', 'spr'])
ephemerisGE = namedtuple('ephemeris', ['sv', 'tow', 'flags', 
                             'iodc', 'toc', 'ura',
                             'healthS', 'wn', 'tgd', 
                             'af2', 'af1', 'af0', 'toe', 
                             'iode', 'rootA', 'ecc', 'm0', 
                             'omega0', 'inc0', 'argPer', 
                             'deln', 'omegaDot', 'incDot', 
                             'crc', 'crs', 'cuc', 'cus', 'cic', 'cis'])
ephemerisNE = namedtuple('ephemerisNE', ['sv', 'frqNum', 'dne',
                         'tk', 'tb', 'health', 'age', 'flags',
                         'rx', 'ry', 'rz', 'vx', 'vy', 'vz', 'wx', 'wy', 'wz',
                         'tauSys', 'tau', 'gamma', 'fDelTauN', 'nFt', 'nN4', 'flags2', 'cs'])

In [7]:
def StatusGPS(sys, sys0,  x0 = 0, y0 = 0, z0 = 0):
    Status = []
    for epoch in tnrange(len(sys0)):
        stat = []
        #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
        rt = sys[epoch]['RT']
        pv = RKSa[0]['receiver']
        pv0 = sys[0]['receiver']
        sats = sys0[epoch]#['pseudo_params']
        for usi in sv_gps:
            if sats[usi].usi in EphemerisGE.keys():
                cur_eph = EphemerisGE[sats[usi].usi]
                X,Y,Z = Uchet(*gps_coord(rt, cur_eph, sats[usi].spr), pv.x+x0,pv.y+y0,pv.z+z0)
                az, el = np.arctan2(X,Y)*180/np.pi, np.arctan2(Z,Y)*180/np.pi
                R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
                stat.append(status(rt, sats[usi].usi, sats[usi].usi, X,Y,Z,pv0.x,pv0.y,pv0.z, R,(sats[usi].scp),sats[usi].spr, az, el))
            else:
                stat.append(status(rt, sats[usi].usi, sats[usi].usi, 0,0,0,0,0,0, 0,(sats[usi].scp),sats[usi].spr, 0, 0))
        Status.append({rec.usi: rec for rec in stat if rec.X != 0})
    return Status

def StatusGLN(sys, sys0,  x0 = 0, y0 = 0, z0 = 0):
    Status = []
    for epoch in tnrange(len(sys0)):
        stat = []
        #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
        rt = sys[epoch]['RT']
        nt = sys[epoch]['NT']
        pv = RKSa[epoch]['receiver']
        sats = sys0[epoch]#['pseudo_params']
        for usi in sv_gln:
            if sats[usi].usi in EphemerisNE.keys():
                cur_eph = EphemerisNE[sats[usi].usi][select_eph(nt, EphemerisNE[sats[usi].usi].keys())]
                sv = [*EphemerisNE[sats[usi].usi].values()][0].sv
                X,Y,Z = Uchet(*gln_coord(nt, 0, cur_eph, sats[usi].spr), pv.x, pv.y, pv.z)
                az, el = np.arctan2(X,Y)*180/np.pi, np.arctan2(Z,Y)*180/np.pi
                R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
                stat.append(status(rt, sats[usi].usi, sv, X, Y, Z, pv.x + x0, pv.y + y0, pv.z + z0, R, sats[usi].scp, sats[usi].spr, az, el))
            else:
                stat.append(status(rt, sats[usi].usi, 0, 0,0,0,0,0,0, 0,(sats[usi].scp),sats[usi].spr, 0, 0))
        Status.append({rec.usi: rec for rec in stat if rec.X != 0})
    return Status

In [8]:
def pick_elem(dict):
    return [dict[b'AN'].channel, dict[b'SI'].usi, dict[b'pc'].scp, dict[b'rc'].spr ]

def process_epoch(epoch, chan = 0):
    ep = [map_rec(general_parser(rec)) for rec in epoch]
    ep_dict = {rec.id: rec for rec in ep}
    processed_data = {
        'RT': (ep_dict[b'~~'].tod[0]*1e-3)%86400,
        'GT': ep_dict[b'GT'],
        'NT': (ep_dict[b'NT'].tod*1e-3)%86400,
        'pseudo_params': [pseudo_params(channel,usi,(scp/1024.),(spr*1e-11+0.075)) for channel,usi,scp,spr in zip( *pick_elem(ep_dict)) if channel in chan],
        'receiver': ep_dict[ b'PV'],
        'NN': ep_dict[b'NN'],
    }
    if b'GE' in ep_dict:
        processed_data['ephemerisGE'] = ep_dict[ b'GE']
    elif b'NE' in ep_dict:
            processed_data['ephemerisNE'] = ep_dict[ b'NE']
    return processed_data

def find_sat(channel, usi, sats):
    if channel == 0:
        return [[sat for sat in sats if (sat.usi == usi)] , pseudo_params(channel, usi, 0, 0)]
    else:
        return next( (sat for sat in sats if (sat.channel == channel and sat.usi == usi)) , pseudo_params(channel, usi, 0, 0))
fR = lambda xr,yr,zr: np.sqrt((xr)**2+(yr)**2+(zr)**2)

def popr(channel):
    if b'b' in channel:
        return 1.39394, 0.79917, -1.14693  
    elif b'c' in channel:
        return 1.219, -0.36045, -0.61941  
    elif b'd' in channel:
        return 0.15215, 1.30038, -0.66836 

def select_eph(target, arr):
    compare = lambda x: abs(target - x)
    return sorted(arr, key=compare)[0]

In [9]:
status = namedtuple('status',['rt','usi', 'sv' ,'X', 'Y', 'Z', 'x', 'y', 'z' ,'R','scp', 'spr' ,'az', 'el'])
status0 = namedtuple('status',['rt', 'usi', 'cp', 'R'])
d_x = namedtuple('d_x',['d_fi','d_R'])
dd_x = namedtuple('dd_x',['dd_fi','dd_R'])

d_x0 = namedtuple('d_x',['d_fi','d_R', 'd_p'])
dd_x0 = namedtuple('dd_x',['dd_fi','dd_R', 'dd_p'])

In [10]:
sv_gps = [*range(1,39)]
sv_gln = [*range(38,70)]
sv_arr = [4,5,13,16,21,26]
#sv_gln00 = [38, 42, 45, 46, 47, 48, 50, 51]
#sv_arr = [26]
channel = b'b'
RKSa = [process_epoch(epoch, b'a') for epoch in epoch_min_arr]
RKSb = [process_epoch(epoch, channel) for epoch in epoch_min_arr]

F = 1575.42e6
C = 299792458.
lam = C / F
#1598.0625

epne = [sat['ephemerisNE'] for sat in RKSa if 'ephemerisNE' in sat]
epge = [sat['ephemerisGE'] for sat in RKSa if 'ephemerisGE' in sat]

EphemerisGE = {rec.sv: rec for rec in [sat['ephemerisGE'] for sat in RKSa if 'ephemerisGE' in sat]}
EphemerisNE = {rec.frqNum+45: {ep.tb: ep for ep in epne if ep.frqNum == rec.frqNum} for rec in epne}


In [11]:
RKSa[0]['NN']

GT(id=b'NN', len=37, osn=(14, 22, 15, 23, 7, 13, 24, 8, 1, 14, 22, 15, 23, 7, 24, 8, 13, 1, 14, 13, 22, 15, 23, 24, 7, 8, 1, 14, 13, 22, 23, 24, 15, 8, 7, 1), cs=118)

In [12]:
a_gps = {epoch: {usi: find_sat(b'a', usi, RKSa[epoch]['pseudo_params']) for usi in sv_gps} for epoch in range(len(RKSa))}
b_gps = {epoch: {usi: find_sat(channel, usi, RKSb[epoch]['pseudo_params']) for usi in sv_gps} for epoch in range(len(RKSa))}

a_gln = {epoch: {usi: find_sat(b'a', usi, RKSa[epoch]['pseudo_params']) for usi in sv_gln} for epoch in range(len(RKSa))}
b_gln = {epoch: {usi: find_sat(channel, usi, RKSb[epoch]['pseudo_params']) for usi in sv_gln} for epoch in range(len(RKSa))}

In [13]:
%%time
dx,dy,dz = popr(channel)

stAgps = StatusGPS(RKSa, a_gps)
stBgps = StatusGPS(RKSb, b_gps, dx,dy,dz)


Wall time: 4.5 s


In [14]:
# %%time
# dx,dy,dz = popr(channel)

# stAgln = StatusGLN(RKSa, a_gln)
# stBgln = StatusGLN(RKSb, b_gln, dx,dy,dz)

In [15]:
# IndexGPS = lambda epoch: next(stAgps[epoch][sv].usi for sv in [*stAgps[epoch].keys()] if int(stAgps[epoch][sv].az) and int(stAgps[epoch][sv].el) in [*range(30,50)])
# IndexGLN = lambda epoch: next(stAgln[epoch][sv].usi for sv in [*stAgln[epoch].keys()] if int(stAgln[epoch][sv].az) and int(stAgln[epoch][sv].el) in [*range(30,50)])
# # elgps = IndexGPS(0)
# # elgln = IndexGLN(0)

In [35]:
A = [4, 5, 13, 15, 16, 20, 21, 26, 29]
Range = range(len(RKSa))
dX = {epoch: {sv: d_x(stBgps[epoch][sv].scp - stAgps[epoch][sv].scp, stBgps[epoch][sv].R - stAgps[epoch][sv].R) for sv in A} for epoch in range(len(RKSa))}

ddM = {epoch: {sv: (dX[epoch][sv].d_fi - dX[epoch][sv].d_R/lam) for sv in A} for epoch in range(len(RKSa))}

ddMsv = {sv: [ddM[epoch][sv]-ddM[0][sv] for epoch in range(len(ddM))] for sv in A}
ddMx = {sv: [ddM[epoch][sv] for epoch in range(len(ddM))] for sv in A}

In [36]:
h_x = namedtuple('h', ['hx', 'hy', 'hz'])
dh = {epoch: {sv: h_x(((stAgps[epoch][sv].X - stAgps[epoch][sv].x) / stAgps[epoch][sv].R) / lam,
                      ((stAgps[epoch][sv].Y - stAgps[epoch][sv].y) / stAgps[epoch][sv].R) / lam,
                      ((stAgps[epoch][sv].Z - stAgps[epoch][sv].z) / stAgps[epoch][sv].R) / lam,) for sv in A} for epoch in Range}

In [37]:
H = {epoch: np.matrix([(dh[epoch][sv].hx, dh[epoch][sv].hy, dh[epoch][sv].hz) for sv in A]) for epoch in Range}

In [38]:
M = {epoch: np.matrix([(dX[epoch][sv].d_fi - dX[epoch][sv].d_R/lam) - (dX[0][sv].d_fi - dX[0][sv].d_R/lam) for sv in A]).T for epoch in Range}

In [39]:
te = 50
((H[te].T * H[te]).I * H[te].T * M[te]).tolist()

[[0.0002632394793911387], [0.0021958352574743575], [0.002118474350348162]]

In [40]:
dT = {te: (H[te].T * H[te]).I * H[te].T * M[te] for te in Range}

In [46]:
dT[te][0].tolist()[0][0]

0.0002632394793911387

In [55]:
np.matrix(popr(b'b')).T

matrix([[ 1.39394],
        [ 0.79917],
        [-1.14693]])

In [56]:
q = figure(width = 1000, height = 400,)
q.line(Range, [(H[te] * np.matrix(popr(b'b')).T).tolist()[0][0] for te in Range])
q.line(Range, [H[te] * np.matrix(popr(b'b')).T.tolist()[0][0] for te in Range], color = 'red')
q.line(Range, [H[te] * np.matrix(popr(b'b')).T.tolist()[0][0] for te in Range], color = 'green')
#q.yaxis.bounds = (-0.3, 0.3)
show(q)

In [ ]:
qqq

In [ ]:
# lam_gln = {usi: C / ((1598.0625 + (usi - 45) * 0.5625)*1e6) for usi in sv_gln}
# F_gln = {usi: ((1598.0625 + (usi - 45) * 0.5625)*1e6) for usi in sv_gln}

In [ ]:
# B = [38, 42, 45, 46, 47, 48, 50, 51]

# Range = range(len(RKSa))
# #Range = range(1300,2100)

# dX0 = {epoch: {sv: d_x0(stBgln[epoch][sv].scp - stAgln[epoch][sv].scp,
#                         stBgln[epoch][sv].R  - stAgln[epoch][sv].R,
#                         stBgln[epoch][sv].spr*C - stAgln[epoch][sv].spr*C) for sv in B} for epoch in Range}

# el = 38

# ddM0 = {epoch: {sv: dX0[epoch][sv].d_fi - dX0[epoch][sv].d_R / lam_gln[sv] + dX0[epoch][el].d_R / lam_gln[el] + (1 / (len(B) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX0[epoch][svm].d_R - dX0[epoch][svm].d_p for svm in B]) for sv in B if sv != el} for epoch in Range}

# ddMsv0 = {sv: [ddM0[epoch][sv]-ddM0[0][sv] for epoch in Range] for sv in B if sv != el}

In [ ]:
# d = figure(plot_width = 1000, plot_height = 400,)
# for sv, color in zip(B, Category10[10]):
#     d.line(Range, [val[sv].d_fi - val[sv].d_R / lam_gln[sv] + val[el].d_R / lam_gln[el]  - val[sv].d_p for key, val in dX0.items()], legend=str(sv), color = color)
# show(d)

In [34]:
p = figure(plot_width = 1000, plot_height = 400,  title="Неопределенное целое b1-"+filename[5:7], 
           x_axis_label='Эпоха', y_axis_label='М',
           tools=TOOLS, toolbar_location="below",)

for sv, color in zip(A, Category10[10]):
    #if sv == el: continue
    p.line(Range, signal.medfilt(ddMsv[sv]),
           color = color, alpha = 0.8, muted_color=color, muted_alpha=0.2, legend=str(sv), line_width = 1)
p.legend.click_policy="mute"
p.legend.orientation = "horizontal"
#p.xaxis.bounds = (0, 1200)
show(p)

NameError: name 'ddMsv' is not defined

In [ ]:
te = 60
sv = 5

In [ ]:
Rc = lambda epoch, sv: stAgps[epoch][sv].R

In [ ]:
hx =  lambda epoch, sv: (stAgps[epoch][sv].X -  stAgps[epoch][sv].x) / Rc(epoch, sv)
hy =  lambda epoch, sv: (stAgps[epoch][sv].Y -  stAgps[epoch][sv].y) / Rc(epoch, sv)
hz =  lambda epoch, sv: (stAgps[epoch][sv].Z -  stAgps[epoch][sv].z) / Rc(epoch, sv)

In [ ]:
def HH(H, dm):
    A = Hfi.T*Hfi
    B = Hfi.T
    return np.linalg.lstsq(A,B*dm,rcond=None)[0]

In [ ]:
Hfi = np.matrix([(hx(te, sv)/ lam, hy(te, sv)/ lam, hz(te, sv)/ lam ) for sv in [4, 5, 13, 15, 16, 20, 21, 26, 29]])
Hfi

In [ ]:
dm = np.matrix([val for val in ddM[te].values()]).T
dm

In [ ]:
np.linalg.lstsq(Hfi.T * Hfi, Hfi.T * dm,rcond=None)[0]

In [ ]:
QQ = []
for te in range(1000):
    Hfi = np.matrix([(hx(te, sv)/ lam, hy(te, sv)/ lam, hz(te, sv)/ lam ) for sv in [4, 5, 13, 15, 16, 20, 21, 26, 29]])

    dm = np.matrix([val for val in ddM[te].values()]).T
    
    QQ.append(np.array(HH(Hfi, dm)))

In [ ]:
d = figure(plot_width = 800, plot_height = 250)
d.line(range(1000), [QQ[ep][0][0] - QQ[0][0][0] for ep in range(1000)])
d.line(range(1000), [QQ[ep][1][0] - QQ[0][1][0] for ep in range(1000)], color = 'red')
d.line(range(1000), [QQ[ep][2][0] - QQ[0][2][0] for ep in range(1000)], color = 'green')
show(d)

In [ ]:
a

In [ ]:
# sv_arr = lambda svs, sys: [sv for sv in svs if all([sys[epoch][sv].scp != 0 for epoch in sys.keys()])]
# sv_arr0 = sv_arr(sv_gps, a_gps)[:-1]
# sv_arr1 = sv_arr(sv_gln, a_gln)

# sv_gln0 = [sv for sv in sv_arr1 if a_gln[0][sv].sv in  EphemerisNE.keys()]
# sv_gln0

# sv_all = lambda svs, sys: [sv for sv in svs if all([sys[epoch][sv].scp != 0 for epoch in sys.keys()])]
# sv_any = lambda svs, sys: [sv for sv in svs if any([sys[epoch][sv].scp != 0 for epoch in sys.keys()])]
# sv_0 = sv_any(sv_gps, a_gps)
# sv_1 = sv_any(sv_gln, a_gln)

# sorted([*EphemerisNE.keys()])
# # спутники для которых есть эфемериды

# sv_arr0 = [4, 5, 13, 15, 16, 20, 21, 26, 29]
# sv_arr0 = [4,5,13,16,21,26]

# %matplotlib inline
# fig = plt.figure(figsize=(18,4))
# #plt.axis([1275, 2150, 0, 1.4e8])
# for sv in sv_arr1:
#     #plt.plot([rgln[epoch][sv].cp for epoch in rgln.keys()], label = sv)
#     plt.plot([a_gln[epoch][sv].scp for epoch in a_gln.keys()], label = sv)
# plt.legend()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from ipywidgets import interact
# import colorcet as cc
# palette = [cc.rainbow[i*int(256/len(sv_gps))] for i in range(len(sv_gps))]
# def f(sv):
#     for sv in [sv]:
#         plt.figure(figsize=(18,4))
#         plt.axis([-100,1.1*len(ddM),-0.25,0.25])
#         plt.plot([ddM[epoch][sv]-ddM[0][sv] for epoch in range(len(ddM))], label = sv)  
#         plt.grid(True)
# interact(f, sv=(1,37))

In [ ]:
a

In [ ]:
# dX = [[ d_x( sat_b1.sv, (sat_r1.scp - sat_b1.scp), (sat_r1.R - sat_b1.R) ) 
#        for sat_b1, sat_r1 in zip(stA[epoch], stB[epoch])] 
#       for epoch in range(len(RKSa))]

# ddM = [[dd_M(ep.sv, (ep.d_fi - ep.d_R / (lam))) for ep in epoch] for epoch in dX]

In [ ]:
stA[0]

In [ ]:
c = 0
for i in range(len(sv_arr)):
    c+=1
    plt.plot([sat[i].dd_M for sat in ddM])

In [ ]:
plt.figure(figsize=(18,4))
# for i in range(len(sv_arr)):
i = 0
plt.plot([ddM[epoch][i] - ddM[0][i]  for epoch in range(len(ddM))], label = sv)
plt.legend()
plt.grid(True)
plt.xlabel(u'Эпохи')
plt.title(u'Целое M')

In [ ]:
a

In [ ]:
MMM = [MM[i]/MM[i-1] for i in range(1,len(MM))]

In [ ]:
from scipy import signal

filtred_data = signal.medfilt(ТВОИ ДАННЫЕ)

In [ ]:
m = signal.medfilt(MM)

In [ ]:
%matplotlib inline
plt.figure(figsize=(18,4))
plt.plot(MM)
plt.plot(m)
plt.grid(True)

In [ ]:
b1_COR = np.array([[[param.X, param.Y, param.Z] for param in sat] for sat in stA])
b1_cor = np.array([[[param.x, param.y, param.z] for param in sat] for sat in stA])

In [ ]:
%matplotlib widget

epoch = 500

x,y,z = [*zip(*b1_cor[epoch])]
X,Y,Z = [*zip(*b1_COR[epoch])]
coords = stA[epoch]
import colorcet as cc
#palette = [cc.rainbow[i*int(256/len(X))] for i in range(len(X))]
fig = plt.figure(figsize=(12,12))
ax = fig.gca(projection='3d')
ax.set_aspect("equal")

Re = 6363032
Rc = 4*Re

# draw cube
r = [-Rc, Rc]
for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
        ax.plot3D(*zip(s, e), color="b", lw = 0.1)

# draw sphere
u, v = np.mgrid[0:2*np.pi:10j, 0:np.pi:10j]
x0 = Re*np.cos(u)*np.sin(v)
y0 = Re*np.sin(u)*np.sin(v)
z0 = Re*np.cos(v)
ax.plot_wireframe(x0, y0, z0, color="k", lw = 0.3)

#draw a point
ax.scatter([0], [0], [0], color="k", s=50)

ax.scatter(x[0],y[0],z[0], s = 30, color = 'red')
ax.text(x[0],y[0],z[0], 'PRM', color='red')
prm_vec = Arrow3D([0, x[0]], [0, y[0]], [0, z[0]], mutation_scale=20, lw=1, arrowstyle="-|>", color="r")
ax.add_artist(prm_vec)

for i in range(len(sv_arr)):
    ax.scatter(X[i],Y[i],Z[i], s = 10)
    ax.text(1.1*X[i],1.1*Y[i],1.1*Z[i], str(coords[i].sv))
    a = Arrow3D([x[i], X[i]], [y[i], Y[i]], [z[i], Z[i]], mutation_scale=10, arrowstyle="-|>", lw = 0.3)
    ax.add_artist(a)


def m_ax(val, XS = 0, YS = 0, ZS = 0, mut = 10, l = 1, c = 'k'):
    x_ = Arrow3D([XS, val+XS], [YS, YS], [ZS, ZS], mutation_scale=mut, lw=l, arrowstyle="-|>", color=c)
    y_ = Arrow3D([XS, XS], [YS, val+YS], [ZS, ZS], mutation_scale=mut, lw=l, arrowstyle="-|>", color=c)
    z_ = Arrow3D([XS, XS], [YS, YS], [ZS, val+ZS], mutation_scale=mut, lw=l, arrowstyle="-|>", color=c)
    ax.text(val+XS, YS, ZS, 'X', color = c)
    ax.text(XS, val+YS, ZS, 'Y', color = c)
    ax.text(XS, YS, val+ZS, 'Z', color = c)
    ax.add_artist(x_)
    ax.add_artist(y_)
    ax.add_artist(z_)

el = 45
xm = Re/np.tan(el)
ym = Re/np.tan(el)
zm = Re   
    
m_ax(Rc)
m_ax(Re, x[0],y[0],z[0],  c = 'r')

In [ ]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.gca(projection='3d')
# ax.set_aspect("equal")
# a,b = 3,4
# q1 = Arrow3D([0, 3], [0, 4], [0, 0], arrowstyle="-|>",mutation_scale = 10, color = 'r', lw = 2)
# q2 = Arrow3D([0, 4], [0, 3], [0, 0], arrowstyle="-|>",mutation_scale = 10, color = 'g', lw = 2)
# q3 = Arrow3D([0, 0.5], [0, 0.5], [0, 0], arrowstyle="-|>",mutation_scale = 10, color = 'b', lw = 2)
# ax.add_artist(q1)
# ax.add_artist(q2)
# ax.add_artist(q3)
# m_ax(1)

In [ ]:
coords

In [ ]:
r = np.sqrt(x**2 + y**2)
tetta = np.arctan2(z,r) 
phi = np.arctan2(y,x)

R = np.sqrt(Y**2 + X**2)
Tetta = np.arctan2(Z,R) 
Phi = np.arctan2(Y,X)

# широта
lat = tetta * 180./np.pi 
# долгота
lng = phi * 180./np.pi

# широта
Lat = Tetta * 180./np.pi 
# долгота
Lng = Phi * 180./np.pi


print(lat[0], lng[0])
print(Lat[0], Lng[0])

F = np.arctan( np.cos(lng - Lng) * np.cos(lat - 0.151) / np.sqrt( 1 - np.cos( 2 * (lng - Lng)) * np.cos( 2 * lat) ) ) * 180./np.pi

svs = [sat.sv for sat in coords]

[*zip(svs, F)]

In [ ]:
a = [1,2,3]
b = [6,5,4]
c = [*zip(a,b)]

# Упорядочим по значениям и ключам.
sorted(c, key=lambda item: (item[1]))

In [ ]:
c